## Calling R from Python

This notebook demonstrates 3 different ways of running R functions from Python: 
1. Self-encapusalted scripts via the `%%R` magic command within a Jupyter notebook
2. Passing pandas DataFrames into R functions and getting results back into Python within a Jupyter notebook
3. Using wrapper Python functions to call R functions behind the scences

### Contents

* [Environment Setup](#Environment-Setup)  
  * [Load R Packages](#Load-R-Packages)  
  * [Load Python Packages](#Load-Python-Packages)  
* [Example: Mixed Effects Models](#Example:-Mixed-Effects-Models)  
  * [Using Python's statsmodels package](#Using-Python's-statsmodels-package)  
  * [Using Rmagic command](#Using-Rmagic-command)  
  * [Running an R script with Python data](#Running-an-R-script-with-Python-data)  
  * [Running an R script outside of Jupyter](#Running-an-R-script-outside-of-Jupyter)  

### Environment Setup

#### Load R Packages

In [1]:
%load_ext rpy2.ipython

In [3]:
%%R
renv::restore()  # run this if the project is not yet in sync with the renv snapshot

library(lme4)
library(geepack)
library(lmerTest)

#### Load Python Packages

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from fit_lmem import compute_estimates_and_cis

#### Example: Mixed Effects Models

This example is courtesy of [Jin Hyun Cheong](https://jinhyuncheong.medium.com/) and his excellent [towards data science post](https://towardsdatascience.com/how-to-run-linear-mixed-effects-models-in-python-jupyter-notebooks-4f8079c4b589). The motivation is to show that we can get the same results using a Python implementation of mixed effects models as we would with R. Then, we can take that one step further and do things with R that are not available in Python in this setting.

In [7]:
# Load dataset
data = sm.datasets.get_rdataset('dietox', 'geepack').data
data.shape

(861, 8)

##### Using Python's `statsmodels` package

In [8]:
# Fit a simple mixed effects model using statsmodels implementation in Python
model_definition = smf.mixedlm(
    formula='Weight ~ Time', 
    data=data, 
    groups=data['Pig'], 
    re_formula='~Time'
)

model = model_definition.fit(
    method=['lbfgs']
)

print(model.summary())

           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  Weight    
No. Observations:  861      Method:              REML      
No. Groups:        72       Scale:               6.0372    
Min. group size:   11       Log-Likelihood:      -2217.0475
Max. group size:   12       Converged:           Yes       
Mean group size:   12.0                                    
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        15.739    0.550 28.603 0.000 14.660 16.817
Time              6.939    0.080 86.925 0.000  6.783  7.095
Group Var        19.503    1.561                           
Group x Time Cov  0.294    0.153                           
Time Var          0.416    0.033                           



##### Using `Rmagic` command

In [9]:
%%R
data <- data(
    dietox, 
    package='geepack'
)

model <- lmer(
    'Weight ~ Time + (1 + Time | Pig)', 
    data=dietox
)

model_summary = summary(model)

print(model_summary)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: "Weight ~ Time + (1 + Time | Pig)"
   Data: dietox

REML criterion at convergence: 4434.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4286 -0.5529 -0.0416  0.4841  3.5624 

Random effects:
 Groups   Name        Variance Std.Dev. Corr
 Pig      (Intercept) 19.4929  4.415        
          Time         0.4161  0.645    0.10
 Residual              6.0375  2.457        
Number of obs: 861, groups:  Pig, 72

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)    
(Intercept) 15.73865    0.55012 71.03978   28.61   <2e-16 ***
Time         6.93901    0.07983 71.06517   86.93   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
     (Intr)
Time 0.005 


We can see that we get exactly the same results as we do using Python's `statsmodels`.

##### Running an R script with Python data

Now, here's what we really came here for - making Python and R work together. We are going to take a pandas DataFrame from Python, pass it to R and fetch the result back, so we can use it in the Python world again.

In [10]:
# Load dataset as a pandas DataFrame as before
data = sm.datasets.get_rdataset('dietox', 'geepack').data

print(type(data))  # nothing up our sleves - this is a normal pandas DataFrame
print(data.shape)  # same shape as before

<class 'pandas.core.frame.DataFrame'>
(861, 8)


In [11]:
formula = 'Weight ~ Time + (1 + Time | Pig)'  # same model definition as we used in the example above

Using the `Rmagic` command we will input the data and the model definition via `-i data,formula` arguments, and output coefficients via `-o coeffs` argument.

In [12]:
%%R -i data,formula -o coeffs

model <- lmer(
    formula, 
    data=data
)

coeffs <- coef(summary(model))

In [13]:
type(coeffs)

numpy.ndarray

Now we have estimated coefficients as a numpy array and can work with them the way we're used to.

In [14]:
intercept_coeff = coeffs[0][0]
time_coeff = coeffs[1][0]

print(intercept_coeff)  
print(time_coeff)

15.738650177945749
6.939014133855649


##### Running an R script outside of Jupyter

One final step is to be able to run R scripts and Python scripts seamlessly together. In the example below, we first define an R function to fit a linear mixed effects model in `fit_lmem.R`, and then we create a Python wrapper function in `fit_lmem.py`. To make everything work together we need to add the following steps to the Python script: 

1. Import the R function that will do the model estimation 

```python
import rpy2.robjects as ro
ro.r['source']('fit_lmem.R')

lmem_r_function = ro.globalenv['compute_estimates_and_conf_ints']
```

2. Transform pandas DataFrame to R-friendly format

```python
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

with localconverter(ro.default_converter + pandas2ri.converter):
    data_r = ro.conversion.py2rpy(data)
```

3. Call the R function to compute the estimates

```python
r_estimates = lmem_r_function(
        data=data_r,
        formula=formula,
        ci_method=ci_method,
    )
```

4. Format and return results as a pandas DataFrame
```python
return DataFrame([
    ...
])
```

In [15]:
estimates = compute_estimates_and_cis(
    data=data, 
    formula=formula, 
    ci_method='profile'
)

In [17]:
estimates

,Name,Estimate,95% CI,P-Value
0,Intercept,15.739,"(14.653, 16.824)",0.0
1,Time,6.939,"(6.782, 7.096)",0.0


And voila, we get parameter estimates, 95% confidence intervals and p-vales all using Python syntax and R smarts behind the scenes.